In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.0f' % x)
import plotly.express as px

## 數據前處理階段

In [ ]:
# sales = pd.read_excel('/Users/kai/Desktop/Weekly report_v0.1.xlsx', sheet_name='出貨明細') # Mac使用
sales = pd.read_excel(r'C:\Users\kaihsu\Desktop\業績總表\Weekly report_v0.1.xlsx', sheet_name='出貨明細')
# budget = pd.read_excel('/Users/kai/Desktop/Weekly report_v0.1.xlsx', sheet_name='2020預算', usecols='A:F') # Mac使用
budget = pd.read_excel(r'C:\Users\kaihsu\Desktop\業績總表\Weekly report_v0.1.xlsx', sheet_name='2020預算', usecols='A:F')

In [ ]:
budget.insert(2, 'BU', value=budget.apply(lambda x: 'RF' if 'RF' in x['客戶名稱'] else 'Component', axis=1))

In [ ]:
budget['客戶名稱'] = budget['客戶名稱'].str.replace(' RF', '')

In [ ]:
client_dict = {'Bourns': 'Bourns', 'Eaton': 'Eaton', 'Laird': 'Laird', 'LF': 'LF', 'LF-OEM': 'LF ORM', 'WE': 'WE',
               '信昌': '信昌', '東莞弘電': '東莞弘電', '藝感': '藝感', '華科': '華新科技',
               'Mt System(Component)': 'MT-SYSTEM', 'Teltonika(Component)': 'TELTONIKA', 'Mt System(RF)': 'MT-SYSTEM',
               'Teltonika(RF)': 'TELTONIKA', 'Te(RF)': 'TE', 'Credence(India)': 'Credence',
               'Millenium(India)': 'Millenium', 'Nexty(India)': 'Nexty', 'OSE(India)': 'OSE', 'Others/One Time PO(India)': 'Others/One Time PO(India)'}
budget['客戶名稱'] = budget['客戶名稱'].map(client_dict)

In [ ]:
sales = sales[~sales['狀態'].str.contains('待通知 | 合約')]

## Treemap

In [ ]:
test = sales.groupby(['預交年份', '負責業務', 'Group', 'BG', '預交月份'])[['數量', '本國幣別NTD']].sum().reset_index()

In [ ]:
fig = px.treemap(test, path=['預交年份', '負責業務', 'Group', 'BG', '預交月份'], values='本國幣別NTD',
                color='數量', color_continuous_scale='RdBu'
                )
fig.show()

In [ ]:
test1 = test.query('Group == ["TELTONIKA", "MT-SYSTEM"]')
fig = px.treemap(test1, path=['預交年份', 'Group', 'BG'], values='本國幣別NTD',
                color='數量', color_continuous_scale='RdBu'
                )
fig.show()

## Line plot

In [ ]:
line_plot = test.groupby(['預交年份', '預交月份', '負責業務'])[['本國幣別NTD']].sum().reset_index()
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July',
               'August', 'September', 'October', 'November', 'December']
line_plot['預交月份'] = pd.Categorical(line_plot['預交月份'], ordered=True, categories=month_order)
line_plot = line_plot.sort_values(by='預交月份')

In [ ]:
line = px.line(line_plot, x='預交月份', y='本國幣別NTD', color='預交年份', category_orders={'預交月份':month_order}, facet_col='負責業務')
line.show()

## Bar plot

In [ ]:
df = test.query('預交年份 == (2020, 2019, 2021)')
df = df.groupby(['預交年份', '負責業務', '預交月份'])[['本國幣別NTD']].sum().reset_index()
df['預交年份'] = df['預交年份'].astype(str)
bar = px.bar(df, x='預交月份', y='本國幣別NTD', color='預交年份', barmode='group',
             category_orders={'預交月份':month_order}, facet_col='負責業務'
            )
bar.show()